In [1]:
import os
from glob import glob
import numpy as np
import tensorflow as tf

C:\Users\JoonHo Lee\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [7]:
def _int64_feature(value):
    """Wrapper for inserting int64 features into Example proto."""
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


def _float_feature(value):
    """Wrapper for inserting float features into Example proto."""
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(float_list=tf.train.FloatList(value=value))


def _bytes_feature(value):
    """Wrapper for inserting bytes features into Example proto."""
    if not isinstance(value, list):
        value = [value]
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))

In [2]:
from PIL import Image

dog_paths = glob("data/dog*.jpg")
cat_paths = glob("data/cat*.jpg")

path = dog_paths[0]

def read_image(path):
    image = Image.open(path)
    image = np.array(image)
    return image

In [6]:
tfrecords_name = 'catdog.tfrecords.gz'
options = tf.python_io.TFRecordOptions(tf.python_io.TFRecordCompressionType.GZIP)  # tfrecords로 변환할 때 gz 파일로 압축해준다.
writer = tf.python_io.TFRecordWriter(path=tfrecords_name, options=options)         # 압축 해주지 않으면 raw 하기 때문에 용량이 크다

In [9]:
data_list = glob("data/*.jpg")

for path in data_list:
    if 'cat' in path:
        class_num = 0
    elif 'dog' in path:
        class_num = 1
    
    image = read_image(path)
    raw_image = image.tostring()
    
    string_set = tf.train.Example(features=tf.train.Features(feature={
        'image': _bytes_feature(raw_image),
        'class': _int64_feature(class_num)
    }))
    
    writer.write(string_set.SerializeToString())
    
writer.close()

# read tfdata

In [11]:
tfrecords_path = 'catdog.tfrecords.gz'

In [13]:
dataset = tf.data.TFRecordDataset(tfrecords_path)

In [14]:
def _resize_function(image_decoded, label):
    image_decoded.set_shape([None, None, None])
    image_resized = tf.image.resize_images(image_decoded, [128, 128])
    return image_resize, label